Alex Malz, David Mykytyn

In [ ]:
import numpy as np
import scipy.stats as sps
import scipy.optimize as spo
import matplotlib.pyplot as plt
%matplotlib inline

# Simulate some mock data

In [ ]:
def make_gauss(params=None):
    (amp, loc, scale) = params
    func = sps.norm(loc, scale)
    out = lambda x: amp * func.pdf(x)
    return out


def make_sine(params=None):
    (amp, period, phase) = params
    func = lambda x: amp / 2. * (np.sin(period * x + phase) + 1.)
    return func

In [ ]:
def make_cadence(start, stop, freq, scatter):
    assert(scatter < freq)
    
    perfect = np.arange(start, stop, freq)
    jitter = (np.random.uniform(np.shape(perfect)) - 0.5) * scatter * 2.
    perturbed = perfect + jitter
    return perturbed

In [ ]:
flux_scatter = 0.1

model = make_gauss(params=(50., 100., 10.))
times = make_cadence(0., 200., 5., 0.1)
lc_dat = model(times) + sps.norm(0., flux_scatter).rvs(np.shape(times))

times2 = make_cadence(0., 200., 5., 0.1)
mod2 = make_sine(params=(5., 20., 0.))
dat2 = mod2(times) + sps.norm(0., flux_scatter).rvs(np.shape(times))

In [ ]:
plt.errorbar(times, lc_dat, yerr=flux_scatter*np.ones_like(times), linestyle='None', marker='o')
plt.errorbar(times, dat2, yerr=flux_scatter*np.ones_like(times), linestyle='None', marker='+')

In [ ]:
help(plt.errorbar)

## Permitted transformations

* shiftx
* stretchx
* shifty
* stretchy
* (cross-talk between bands)


## Consistency metric

find MAP/MLE of p(A = B | lc_A, lc_B)
optimize over shift/stretch params

merge (x_A, x_B) and (y_A, y_B)

In [ ]:
def transform((x, y), (deltax, deltay, stretchx, stretchy)):
    new_x = (stretchx * x) + deltax
    new_y = (stretchy * y) + deltay
    return (new_x, new_y)

def merge((xA, yA), (xB, yB)):
    new_x = np.concatenate((xA, xB))
    new_y = np.concatenate((yA, yB))
    order = np.argsort(new_x)
    ord_x = new_x[order]
    ord_y = new_y[order]
    pass

In [ ]:
def connect_the_dots((x, y)):
    x_difs = x[1:] - x[:-1]
    y_difs = y[1:] - y[:-1]
    slopes = y_difs / x_difs
    lengths = x[1:] - x[:-1]
    return np.dot(slopes, lengths)

In [ ]:
def find_max_prob((xa, ya), (xb, yb)):
    
    origa = connect_the_dots((xa, ya))
    origb = connect_the_dots((xb, yb))
    orig_tot = origa + origb
    
    def _helper((deltax, deltay, stretchx, stretchy)):
        ivals = (0., 0., 1., 1.)
        (x, y) = transform((xb, yb), (deltax, deltay, stretchx, stretchy))
        (x_both, y_both) = merge((xa, ya), (x, y)) 
        length = connect_the_dots((x_both, y_both))
        return(length - orig_tot)
    
    res = spo.minimize(_helper, ivals)
    return(res)



In [ ]:
help(spo.minimize)

In [ ]:
ans = find_max_prob((times, lc_dat), (times, lc_dat))

# Reduce to summary statistics

periodogram -- identify periodicity and stochastic noise levels

flux per time bins -- trends keeping bin size constant but changing bin ends

abs/percent change in color and total flux/magnitude

# Cluster in the space of summary statistics

kdtree (and more)

# Other ideas

pairwise combinations/comparisons?